# Tutorial 1: Matching

## 0. Import libraries 

In [3]:
import pandas as pd
import numpy as np
from hypex import Matcher
import warnings 
warnings.simplefilter(action='ignore', category=FutureWarning)

## 1. Create or upload your dataset  
In this case we will create random dataset with known effect size  
If you have your own dataset, go to the part 2 


In [4]:
from hypex.dataset.dataset import Dataset

In [23]:
data = Dataset(na_columns=['feature_col_3', 'feature_col_2'], num_main_causes_cols=2)
data.df

,info_col_1,info_col_2,feature_col_1,feature_col_2,feature_col_3,feature_col_4,treatment_1,outcome_1
0,301,N,male,Deposit,NaN,0.0,1.0,0.424610
1,13294,F,female,NaN,-1.172302,2.0,1.0,1.329602
2,12790,F,male,Credit,0.739650,0.0,1.0,1.259447
3,10966,F,female,Deposit,-1.207812,0.0,1.0,0.156809
4,10534,N,male,Deposit,-0.635289,3.0,0.0,2.316046
...,...,...,...,...,...,...,...,...
4995,9184,F,male,Deposit,-1.316812,1.0,1.0,0.835871
4996,8713,F,male,Credit,-0.043342,1.0,1.0,2.905103
4997,10306,N,female,Deposit,-1.016003,3.0,0.0,1.774634
4998,913,N,male,Investment,0.558224,2.0,0.0,1.425498


In [24]:
data.df.columns

Index(['info_col_1', 'info_col_2', 'feature_col_1', 'feature_col_2',
       'feature_col_3', 'feature_col_4', 'treatment_1', 'outcome_1'],
      dtype='object')

In [25]:
data.df.treatment_1.value_counts()

treatment_1
1.0    2539
0.0    2461
Name: count, dtype: int64

In [26]:
data.df.isna().sum()

info_col_1         0
info_col_2         0
feature_col_1      0
feature_col_2    500
feature_col_3    500
feature_col_4      0
treatment_1        0
outcome_1          0
dtype: int64

## 2. Matching  
### 2.0 Init params
info_col used to define informative attributes that should not be part of matching, such as user_id  
But to explicitly store this column in the table, so that you can compare directly after computation

In [27]:
info_col = data.info_col_names

outcome = data.outcome_name[0]
treatment = data.treatment_name[0]

### 2.1 Simple matching
This is the easiest way to initialize and calculate metrics on a Matching task  
Use it when you are clear about each attribute or if you don't have any additional task conditions (Strict equality for certain features) 

In [41]:
# Standard model with base parameters
model = Matcher(input_data=data.df, outcome=outcome, treatment=treatment, info_col=info_col)
results, quality_results, df_matched = model.estimate()

[21.11.2023 16:29:49 | hypex | INFO]: Number of NaN values filled with zeros: 1000


  0%|          | 0/5000 [00:00<?, ?it/s]

Get untreated index:   0%|          | 0/5000 [00:00<?, ?it/s]

Get treated index:  51%|█████     | 2539/5000 [00:00<00:00, 28702.40it/s]

Get treated index: 100%|██████████| 5000/5000 [00:00<00:00, 25467.10it/s]

Get treated index: 100%|██████████| 5000/5000 [00:00<00:00, 23387.13it/s]


In [42]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,1.879292,0.066988,0.0,1.747996,2.010589,outcome_1
ATC,1.839296,0.075374,0.0,1.691562,1.987029,outcome_1
ATT,1.918060,0.074516,0.0,1.772010,2.064111,outcome_1


In [43]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [44]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
feature_col_3,0.999945,0.995579
feature_col_4,1.000000,1.000000


In [45]:
df_matched

,index,feature_col_3,feature_col_4,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_3_matched,feature_col_4_matched,feature_col_1_male_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched
0,0,0.000000,0.0,1,0,1,0,0.000000,0.0,1.0,0.0,1.0,0.0,"[1600, 1960, 2060, 3100, 4250, 4990]",0.424610,-1.827587,2.252197,1,0
1,1,-1.172302,2.0,0,0,0,0,-1.068343,2.0,0.0,0.0,0.0,0.0,[4461],1.329602,-0.073953,1.470158,1,0
2,2,0.739650,0.0,1,1,0,0,0.728773,0.0,1.0,1.0,0.0,0.0,[2365],1.259447,1.022800,0.229678,1,0
3,3,-1.207812,0.0,0,0,1,0,-1.195921,0.0,0.0,0.0,1.0,0.0,[1966],0.156809,-1.600113,1.764540,1,0
4,6,-1.507264,3.0,1,0,0,1,-1.520801,3.0,1.0,0.0,0.0,1.0,[1947],4.372272,1.890139,2.473461,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,4992,-0.776343,2.0,1,0,0,1,-0.756574,2.0,1.0,0.0,0.0,1.0,[2434],1.076212,4.152876,3.049178,0,1
2457,4993,1.526767,1.0,1,0,0,1,1.549173,1.0,1.0,0.0,0.0,1.0,[4248],1.926302,6.554668,4.597212,0,1
2458,4994,-0.150107,1.0,1,0,1,0,-0.146332,1.0,1.0,0.0,1.0,0.0,[1907],0.101794,3.677723,3.570680,0,1
2459,4997,-1.016003,3.0,0,0,1,0,-1.017882,3.0,0.0,0.0,1.0,0.0,[3978],1.774634,6.768588,4.996567,0,1


In [49]:
df_matched[df_matched['feature_col_1_male'] != df_matched['feature_col_1_male_matched']]

,index,feature_col_3,feature_col_4,feature_col_1_male,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_3_matched,feature_col_4_matched,feature_col_1_male_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched


### 2.2 Matching with a fixed variable  
Used when you have categorical feature(s) that you want to compare by strict equality  
group_col is used for strict comparison of categorical features.  
In our case there is only one attribute  
If there are several such attributes, you should make one of them and use it

In [55]:
group_col = 'feature_col_1'

In [56]:
model = Matcher(input_data=data.df, outcome=outcome, treatment=treatment,
                info_col=info_col, group_col=group_col)
results, quality_results, df_matched = model.estimate()

[21.11.2023 16:33:54 | hypex | INFO]: Number of NaN values filled with zeros: 1000



  0%|          | 0/4 [00:00<?, ?it/s]


Get untreated index by group female:   0%|          | 0/4 [00:00<?, ?it/s]


Get treated index by group female:  25%|██▌       | 1/4 [00:00<00:00, 10.01it/s]


Get treated index by group female:  50%|█████     | 2/4 [00:00<00:00, 13.40it/s]


Get treated index by group female:  50%|█████     | 2/4 [00:00<00:00, 13.40it/s]


Get untreated index by group male:  50%|█████     | 2/4 [00:00<00:00, 13.40it/s]


Get treated index by group male:  75%|███████▌  | 3/4 [00:00<00:00, 13.40it/s]  


Get treated index by group male: 100%|██████████| 4/4 [00:00<00:00,  9.62it/s]


Get treated index by group male: 100%|██████████| 4/4 [00:00<00:00,  9.30it/s]


In [57]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,1.879802,0.066982,0.0,1.748518,2.011086,outcome_1
ATC,1.840331,0.075393,0.0,1.692561,1.988102,outcome_1
ATT,1.918060,0.074493,0.0,1.772054,2.064067,outcome_1


In [58]:
df_matched

,index,feature_col_3,feature_col_4,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_1,feature_col_3_matched,feature_col_4_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,feature_col_1_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched
0,1,-1.172302,2.0,0,0,0,female,-1.068343,2.0,0.0,0.0,0.0,female,[4461],1.329602,-0.073953,1.470145,1,0
1,3,-1.207812,0.0,0,1,0,female,-1.195921,0.0,0.0,1.0,0.0,female,[1966],0.156809,-1.600113,1.764539,1,0
2,8,-1.249708,2.0,0,0,1,female,-1.251400,2.0,0.0,0.0,1.0,female,[2675],3.446066,-0.254556,3.699538,1,0
3,17,-1.135232,1.0,0,1,0,female,-1.145254,1.0,0.0,1.0,0.0,female,[639],0.786651,0.211591,0.568640,1,0
4,19,-0.625021,0.0,1,0,0,female,-0.634467,0.0,1.0,0.0,0.0,female,[929],1.154462,-1.318967,2.467378,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2456,4990,0.000000,0.0,0,1,0,male,0.000000,0.0,0.0,1.0,0.0,male,"[0, 160, 360, 1000, 1300, 1510, 1650, 2270, 41...",-2.339942,-0.001779,2.338163,0,1
2457,4992,-0.776343,2.0,0,0,1,male,-0.756574,2.0,0.0,0.0,1.0,male,[2434],1.076212,4.152876,3.049215,0,1
2458,4993,1.526767,1.0,0,0,1,male,1.549173,1.0,0.0,0.0,1.0,male,[4248],1.926302,6.554668,4.597255,0,1
2459,4994,-0.150107,1.0,0,1,0,male,-0.146332,1.0,0.0,1.0,0.0,male,[1907],0.101794,3.677723,3.570687,0,1


In [59]:
df_matched[df_matched['feature_col_1'] != df_matched['feature_col_1_matched']]

,index,feature_col_3,feature_col_4,feature_col_2_Credit,feature_col_2_Deposit,feature_col_2_Investment,feature_col_1,feature_col_3_matched,feature_col_4_matched,feature_col_2_Credit_matched,feature_col_2_Deposit_matched,feature_col_2_Investment_matched,feature_col_1_matched,index_matched,outcome_1,outcome_1_matched,outcome_1_matched_bias,treatment_1,treatment_1_matched


## 3. Results  
### 3.1 ATE, ATT, ATC

In [60]:
results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,1.879802,0.066982,0.0,1.748518,2.011086,outcome_1
ATC,1.840331,0.075393,0.0,1.692561,1.988102,outcome_1
ATT,1.918060,0.074493,0.0,1.772054,2.064067,outcome_1


### 3.2 SMD, PSI, KS-test, repeats

In [61]:
quality_results.keys()

dict_keys(['psi', 'ks_test', 'smd', 'repeats'])

In [62]:
quality_results['psi']

,column_treated,anomaly_score_treated,check_result_treated,column_untreated,anomaly_score_untreated,check_result_untreated
0,feature_col_1_treated,0.0,OK,feature_col_1_untreated,0.0,OK
1,feature_col_2_Credit_treated,0.0,OK,feature_col_2_Credit_untreated,0.0,OK
2,feature_col_2_Deposit_treated,0.0,OK,feature_col_2_Deposit_untreated,0.0,OK
3,feature_col_2_Investment_treated,0.0,OK,feature_col_2_Investment_untreated,0.0,OK
4,feature_col_3_treated,0.0,OK,feature_col_3_untreated,0.0,OK
5,feature_col_4_treated,0.0,OK,feature_col_4_untreated,0.0,OK


In [63]:
quality_results['ks_test']

,match_control_to_treat,match_treat_to_control
feature_col_3,0.999945,0.993136
feature_col_4,1.000000,1.000000


In [22]:
quality_results['repeats']

{'match_control_to_treat': 0.93, 'match_treat_to_control': 0.09}

### 3.3 Validation
Validates estimated effect:
1. by replacing real treatment (`random_treatment`) with random placebo treatment.
 Estimated effect must be droped to zero;
2. by adding random feature (`random_feature`). Estimated effect shouldn't change
significantly, p-val < 0.05;
3. estimates effect on subset of data (`subset_refuter`) (default fraction is 0.8). Estimated effect
shouldn't change significantly, p-val < 0.05.

In [64]:
model.validate_result(refuter="random_treatment", effect_type="att", n_sim=10)

100%|██████████| 10/10 [00:06<00:00,  1.43it/s]


{'outcome_1': [0.005919635227248263, 0.0]}

## 4. Save model

In [65]:
model.save("test_model.pickle")

In [66]:
model2 = Matcher.load("test_model.pickle")

In [67]:
model2.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,1.879802,0.066982,0.0,1.748518,2.011086,outcome_1
ATC,1.840331,0.075393,0.0,1.692561,1.988102,outcome_1
ATT,1.918060,0.074493,0.0,1.772054,2.064067,outcome_1


In [68]:
model.results

,effect_size,std_err,p-val,ci_lower,ci_upper,outcome
ATE,1.879802,0.066982,0.0,1.748518,2.011086,outcome_1
ATC,1.840331,0.075393,0.0,1.692561,1.988102,outcome_1
ATT,1.918060,0.074493,0.0,1.772054,2.064067,outcome_1
